In [1]:
import zipfile
from google.colab import drive

drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [2]:
!unzip "/content/gdrive/My Drive/data_1.zip"

Archive:  /content/gdrive/My Drive/data_1.zip
   creating: data_1/
   creating: data_1/train/
   creating: data_1/train/healthy/
  inflating: data_1/train/healthy/IMG20180905151016.jpg  
  inflating: data_1/train/healthy/IMG20180905151020.jpg  
  inflating: data_1/train/healthy/IMG20180905151023.jpg  
  inflating: data_1/train/healthy/IMG20180905151026.jpg  
  inflating: data_1/train/healthy/IMG20180905151106.jpg  
  inflating: data_1/train/healthy/IMG20180905151109.jpg  
  inflating: data_1/train/healthy/IMG20180905151114.jpg  
  inflating: data_1/train/healthy/IMG20180905151118.jpg  
  inflating: data_1/train/healthy/IMG20180905151122.jpg  
  inflating: data_1/train/healthy/IMG20180905151135.jpg  
  inflating: data_1/train/healthy/IMG20180905151138.jpg  
  inflating: data_1/train/healthy/IMG20180905151142.jpg  
  inflating: data_1/train/healthy/IMG20180905151145.jpg  
  inflating: data_1/train/healthy/IMG20180905151202.jpg  
  inflating: data_1/train/healthy/IMG20180905151207.jpg  
 

In [3]:
!ls

data_1	gdrive	sample_data


In [4]:
!rm bottleneck_features_train.npy bottleneck_features_validation.npy bottleneck_fc_model.h5

rm: cannot remove 'bottleneck_features_train.npy': No such file or directory
rm: cannot remove 'bottleneck_features_validation.npy': No such file or directory
rm: cannot remove 'bottleneck_fc_model.h5': No such file or directory


In [5]:
!ls data_1/train/healthy | wc -l
!ls data_1/train/defective | wc -l
!ls data_1/test/healthy | wc -l
!ls data_1/test/defective | wc -l

89
89
22
22


In [ ]:
!ls data/train

defective  healthy


In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 512, 512

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data_1/train'
validation_data_dir = 'data_1/test'
nb_train_samples = 176
nb_train_samples_healthy = 89
nb_train_samples_defective = 89

nb_validation_samples = 44
nb_validation_samples_healthy = 22
nb_validation_samples_defective = 22

epochs = 200
batch_size = 22


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy',
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    print(nb_train_samples)
    train_labels = np.array(
        [0] * (nb_train_samples//2) + [1] * (nb_train_samples//2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array(
        [0] * (nb_validation_samples//2) + [1] * (nb_validation_samples//2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

    print(model.summary())


save_bottlebeck_features()
train_top_model()

Using TensorFlow backend.


Found 178 images belonging to 2 classes.
Found 44 images belonging to 2 classes.
176
Train on 176 samples, validate on 44 samples
Epoch 1/200
176/176 [==============================] - 3s 16ms/step - loss: 35.0949 - accuracy: 0.5227 - val_loss: 1.8298 - val_accuracy: 0.6591
Epoch 2/200
176/176 [==============================] - 2s 14ms/step - loss: 8.2764 - accuracy: 0.5625 - val_loss: 8.0954 - val_accuracy: 0.5000
Epoch 3/200
176/176 [==============================] - 3s 14ms/step - loss: 3.0144 - accuracy: 0.6932 - val_loss: 1.1137 - val_accuracy: 0.6591
Epoch 4/200
176/176 [==============================] - 2s 14ms/step - loss: 1.8129 - accuracy: 0.6364 - val_loss: 0.7183 - val_accuracy: 0.6364
Epoch 5/200
176/176 [==============================] - 2s 14ms/step - loss: 0.7391 - accuracy: 0.7216 - val_loss: 0.7161 - val_accuracy: 0.6364
Epoch 6/200
176/176 [==============================] - 3s 14ms/step - loss: 0.5079 - accuracy: 0.7898 - val_loss: 0.8780 - val_accuracy: 0.7045
Epoch

In [ ]:
!ls

bottleneck_fc_model.h5	       bottleneck_features_validation.npy  gdrive
bottleneck_features_train.npy  data				   sample_data


In [2]:
!cp bottleneck_fc_model.h5 "/content/gdrive/My Drive/bottleneck_fc_final_model.h5"